In [1]:
import pandas as pd
import numpy as np

### Import data

In [3]:
data_path = '../../data/MDataFiles_Stage1/'

#Raw data
rankings = pd.read_csv(data_path + 'MMasseyOrdinals.csv')
detailed_reg_season = pd.read_csv(data_path +'MRegularSeasonDetailedResults.csv')
team_coaches = pd.read_csv(data_path + 'MTeamCoaches.csv')
detailed_tourney_results = pd.read_csv(data_path + 'MNCAATourneyDetailedResults.csv')
conf_tourney = pd.read_csv(data_path + 'MConferenceTourneyGames.csv')
team_conferences = pd.read_csv(data_path + 'MTeamConferences.csv')
tourney_seeds = pd.read_csv(data_path + 'MNCAATourneySeeds.csv')
teams = pd.read_csv(data_path + 'MTeams.csv')



### Processing

#### Combine reg season with tournament games

In [4]:
#create flags to indicate if it's a tournament game
detailed_tourney_results['TourneyGame'] = 1
detailed_reg_season['TourneyGame'] = 0

#combine to create full dataset
all_games = pd.concat([detailed_tourney_results, detailed_reg_season])

#### Restructure dataset for predictions

In [5]:
#function to make the dataset complete 
def make_full(df, lose_cols, win_cols, new_cols):
    
    #Remap column names
    df_winners = df[win_cols].rename(columns={i:j for (i,j) in zip(win_cols, new_cols)}).copy()
    df_losers = df[lose_cols].rename(columns={i:j for (i,j) in zip(lose_cols, new_cols)}).copy()
    
    #Add column to remember who won/lost
    df_winners['Outcome'] = 1
    df_losers['Outcome'] = 0
    
    #Encode location 
    df_winners['Loc'] = np.where(df_winners['WLoc'] == 'H', 'H', \
                                 np.where(df_winners['WLoc'] == 'A', 'A', 'N'))
    
    df_losers['Loc'] = np.where(df_losers['WLoc'] == 'H', 'A', \
                                 np.where(df_losers['WLoc'] == 'A', 'H', 'N'))
    
    #Create full df
    full = df_winners.append(df_losers)

    return full

In [6]:
#Need to remap these column names so I can create Team1 and Team2 without encoding any win/loss info
win_cols = ['TourneyGame','Season', 'DayNum', 'WTeamID','LTeamID', 'WScore', 'LScore', 'WLoc', 
'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']

lose_cols = ['TourneyGame','Season', 'DayNum', 'LTeamID', 'WTeamID', 'LScore', 'WScore', 'WLoc',
'NumOT', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3','LFTM', 'LFTA', 'LOR', 'LDR', 
'LAst', 'LTO', 'LStl', 'LBlk', 'LPF','WFGM', 'WFGA', 'WFGM3', 'WFGA3', 
'WFTM', 'WFTA', 'WOR', 'WDR','WAst', 'WTO', 'WStl', 'WBlk', 'WPF']

new_cols = ['TourneyGame','Season', 'DayNum', 'Team1', 'Team2', 'Team1_score', 'Team2_score', 'WLoc',
'NumOT', 'Team1_FGM', 'Team1_FGA', 'Team1_FGM3', 'Team1_FGA3', 'Team1_FTM', 'Team1_FTA', 'Team1_OR', 'Team1_DR',
'Team1_Ast', 'Team1_TO', 'Team1_Stl', 'Team1_Blk', 'Team1_PF', 'Team2_FGM', 'Team2_FGA', 'Team2_FGM3', 'Team2_FGA3',
'Team2_FTM', 'Team2_FTA', 'Team2_OR', 'Team2_DR', 'Team2_Ast', 'Team2_TO', 'Team2_Stl', 'Team2_Blk', 'Team2_PF']


In [7]:
all_games = make_full(all_games, lose_cols, win_cols, new_cols)

In [8]:
all_games.to_csv('output/all_games.csv')

In [9]:
all_games.head()

,TourneyGame,Season,DayNum,Team1,Team2,Team1_score,Team2_score,WLoc,NumOT,Team1_FGM,...,Team2_FTA,Team2_OR,Team2_DR,Team2_Ast,Team2_TO,Team2_Stl,Team2_Blk,Team2_PF,Outcome,Loc
0,1,2003,134,1421,1411,92,84,N,1,32,...,31,17,28,16,15,5,0,22,1,N
1,1,2003,136,1112,1436,80,51,N,0,31,...,7,8,26,12,17,10,3,15,1,N
2,1,2003,136,1113,1272,84,71,N,0,31,...,21,20,22,11,12,2,5,18,1,N
3,1,2003,136,1141,1166,79,73,N,0,29,...,17,14,17,20,21,6,6,21,1,N
4,1,2003,136,1143,1301,76,74,N,1,27,...,20,10,26,16,14,5,8,19,1,N
